First we need to have an access token with Spotify developer account. 

In [57]:
import requests
import base64

# Replace with your own Client ID and Client Secret
CLIENT_ID = 'ab695a06977a4f3d8c16b1eb5570bfe1'
CLIENT_SECRET = 'ab4a0a56466349d38e8ad269131b2f35'

# Base64 encode the client ID and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# Request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Access token obtained successfully.


We can take any playlist from Spotify using Spotipy library, Installing using pip install spotipy.
Then we can make a function that takes in the playlist_id for a Playlist with accesstoken and returns a dataframe with music tracks within that playlist along with all the fields related to each track. Spotify uses track id for unique id. 

In [58]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

def get_trending_playlist_data(playlist_id, access_token):
    # Set up Spotipy with the access token
    sp = spotipy.Spotify(auth=access_token)

    # Get the tracks from the playlist
    playlist_tracks = sp.playlist_tracks(playlist_id, fields='items(track(id, name, artists, album(id, name)))')

    # Extract relevant information and store in a list of dictionaries
    music_data = []
    for track_info in playlist_tracks['items']:
        track = track_info['track']
        track_name = track['name']
        artists = ', '.join([artist['name'] for artist in track['artists']])
        album_name = track['album']['name']
        album_id = track['album']['id']
        track_id = track['id']

        # Get audio features for the track
        audio_features = sp.audio_features(track_id)[0] if track_id != 'Not available' else None

        # Get release date of the album
        try:
            album_info = sp.album(album_id) if album_id != 'Not available' else None
            release_date = album_info['release_date'] if album_info else None
        except:
            release_date = None

        # Get popularity of the track
        try:
            track_info = sp.track(track_id) if track_id != 'Not available' else None
            popularity = track_info['popularity'] if track_info else None
        except:
            popularity = None

        # Add additional track information to the track data
        track_data = {
            'Track Name': track_name,
            'Artists': artists,
            'Album Name': album_name,
            'Album ID': album_id,
            'Track ID': track_id,
            'Popularity': popularity,
            'Release Date': release_date,
            'Duration (ms)': audio_features['duration_ms'] if audio_features else None,
            'Explicit': track_info.get('explicit', None),
            'External URLs': track_info.get('external_urls', {}).get('spotify', None),
            'Danceability': audio_features['danceability'] if audio_features else None,
            'Energy': audio_features['energy'] if audio_features else None,
            'Key': audio_features['key'] if audio_features else None,
            'Loudness': audio_features['loudness'] if audio_features else None,
            'Mode': audio_features['mode'] if audio_features else None,
            'Speechiness': audio_features['speechiness'] if audio_features else None,
            'Acousticness': audio_features['acousticness'] if audio_features else None,
            'Instrumentalness': audio_features['instrumentalness'] if audio_features else None,
            'Liveness': audio_features['liveness'] if audio_features else None,
            'Valence': audio_features['valence'] if audio_features else None,
            'Tempo': audio_features['tempo'] if audio_features else None,
            # Add more attributes as needed
        }

        music_data.append(track_data)

    # Create a pandas DataFrame from the list of dictionaries
    df = pd.DataFrame(music_data)

    return df

In [63]:
playlist_id = '37i9dQZEVXbMDoHDwVN2tF'



# Call the function to get the music data from the playlist and store it in a DataFrame
music_df = get_trending_playlist_data(playlist_id, access_token)

# Display the DataFrame
music_df.head(20)

,Track Name,Artists,Album Name,Album ID,Track ID,Popularity,Release Date,Duration (ms),Explicit,External URLs,...,Energy,Key,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo
0,Espresso,Sabrina Carpenter,Espresso,5quMTd5zeI9yW5UDua8wS4,2qSkIjg1o9h3YT9RAgYN75,96,2024-04-12,175459,True,https://open.spotify.com/track/2qSkIjg1o9h3YT9...,...,0.760,0,-5.478,1,0.0285,0.107000,0.000065,0.1850,0.690,103.969
1,euphoria,Kendrick Lamar,euphoria,32bR4LcEc1PvJEhaKoo4ZN,77DRzu7ERs0TX3roZcre7Q,69,2024-04-30,383639,True,https://open.spotify.com/track/77DRzu7ERs0TX3r...,...,0.643,1,-5.002,1,0.1100,0.046000,0.000000,0.0840,0.142,139.948
2,Fortnight (feat. Post Malone),"Taylor Swift, Post Malone",THE TORTURED POETS DEPARTMENT,1Mo4aZ8pdj6L1jx8zSwJnt,2OzhQlSqBEmt7hmkYxfT6m,94,2024-04-18,228965,False,https://open.spotify.com/track/2OzhQlSqBEmt7hm...,...,0.386,11,-10.976,1,0.0308,0.502000,0.000015,0.0961,0.281,192.004
3,i like the way you kiss me,Artemas,i like the way you kiss me,5HIWDdg3g9CTOtnevKDl1z,2GxrNKugF82CnoRFbQfzPf,100,2024-03-19,142515,False,https://open.spotify.com/track/2GxrNKugF82CnoR...,...,0.946,11,-4.263,1,0.0447,0.000938,0.010600,0.0826,0.747,151.647
4,Gata Only,"FloyyMenor, Cris Mj",Gata Only,5tSQtQGkrCJx3hoQxmLgfM,6XjDF6nds4DE2BBbagZol6,98,2024-02-02,222000,True,https://open.spotify.com/track/6XjDF6nds4DE2BB...,...,0.499,8,-8.472,0,0.0509,0.446000,0.000024,0.0899,0.669,99.986
5,Too Sweet,Hozier,Unreal Unearth: Unheard,3P4SQqMMgjqfqVxgLwtYRk,5Z0UnEtpLDQyYlWwgi8m9C,89,2024-03-22,251424,False,https://open.spotify.com/track/5Z0UnEtpLDQyYlW...,...,0.620,10,-5.505,1,0.0412,0.029500,0.000809,0.0398,0.934,117.038
6,Beautiful Things,Benson Boone,Fireworks & Rollerblades,168CdR21lfn0TTyw1Pkdcm,3xkHsmpQCBMytMJNiDf3Ii,89,2024-04-05,180304,False,https://open.spotify.com/track/3xkHsmpQCBMytMJ...,...,0.471,10,-5.692,1,0.0603,0.151000,0.000000,0.1400,0.219,105.029
7,End of Beginning,Djo,DECIDE,4JDXBZdRPId4devseaAOKH,3qhlB30KknSejmIvZZLjOD,98,2022-09-16,159246,False,https://open.spotify.com/track/3qhlB30KknSejmI...,...,0.454,2,-7.643,1,0.0584,0.035100,0.002590,0.0707,0.912,159.982
8,we can't be friends (wait for your love),Ariana Grande,eternal sunshine,5EYKrEDnKhhcNxGedaRQeK,51ZQ1vr10ffzbwIjDCwqm4,95,2024-03-08,228639,False,https://open.spotify.com/track/51ZQ1vr10ffzbwI...,...,0.646,5,-8.334,1,0.0427,0.061500,0.000030,0.0740,0.295,115.842
9,A Bar Song (Tipsy),Shaboozey,A Bar Song (Tipsy),6egBeCLeGITzGSo5VyRjwZ,2FQrifJ1N335Ljm3TjTVVf,90,2024-04-12,171292,True,https://open.spotify.com/track/2FQrifJ1N335Ljm...,...,0.709,9,-4.950,1,0.0273,0.063300,0.000000,0.0804,0.604,81.012


Data Exploration 

In [40]:
#Check for any null value
import re

#checking for mussing values
print(music_df.isnull().sum()) 

#Cleaning user input
def clean_text(text):
    text = text.strip()
    text = text.replace('.', '').replace('"', '')
    text = re.sub(r'[^\w\s]', '', text)
    return text

Track Name          0
Artists             0
Album Name          0
Album ID            0
Track ID            0
Popularity          0
Release Date        0
Duration (ms)       0
Explicit            0
External URLs       0
Danceability        0
Energy              0
Key                 0
Loudness            0
Mode                0
Speechiness         0
Acousticness        0
Instrumentalness    0
Liveness            0
Valence             0
Tempo               0
dtype: int64


In [41]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from sklearn.metrics.pairwise import cosine_similarity

data = music_df

First, We will use the recency of the tracks by realease to recoomend. Latest songs will be recommended. We will use weight score based on time span since song realeased, inversely propostional to the weight. That means older songs will be scored lesser than newer.
Music Recommender using features like 'Danceability', 'Energy', 'Key', 'Loudness', 'Mode', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo'.

In [43]:
def calculate_weighted_popularity(realease_date):
    #Convert the realese date to datetime object
    realease_date = datetime.strptime(realease_date, '%Y-%m-%d')

    #Calclate the time span between realease date and todays date
    time_span = datetime.now() - realease_date

    #Calculate the weighted popularity score based on timespan
    weight = 1 / (time_span.days + 1)
    return weight

# Normalize the music features using Min-Max scaling
scaler = MinMaxScaler()
music_features = music_df[['Danceability', 'Energy', 'Key', 
                           'Loudness', 'Mode', 'Speechiness', 'Acousticness',
                           'Instrumentalness', 'Liveness', 'Valence', 'Tempo']].values
music_features_scaled = scaler.fit_transform(music_features)


We will make a system that recommends music based on selected audio features. 

In [44]:
# a function to get content-based recommendations based on music features
def content_based_recommendations(input_song_name, num_recommendations=5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get the index of the input song in the music DataFrame
    input_song_index = music_df[music_df['Track Name'] == input_song_name].index[0]

    # Calculate the similarity scores based on music features (cosine similarity)
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    # Get the indices of the most similar songs
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    # Get the names of the most similar songs based on content-based filtering
    content_based_recommendations = music_df.iloc[similar_song_indices][['Track Name', 'Artists', 'Album Name', 'Release Date', 'Popularity']]

    return content_based_recommendations

the function to generate music recommendations based on weighted popularity and combine it with the recommendations of the content-based filtering method using the hybrid approach

In [48]:
# a function to get hybrid recommendations based on weighted popularity
def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):
    if input_song_name not in music_df['Track Name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get content-based recommendations
    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    # Get the popularity score of the input song
    popularity_score = music_df.loc[music_df['Track Name'] == input_song_name, 'Popularity'].values[0]

    # Calculate the weighted popularity score
    weighted_popularity_score = popularity_score * calculate_weighted_popularity(music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0])

    # Combine content-based and popularity-based recommendations based on weighted popularity
    hybrid_recommendations = content_based_rec
    input_song_df = pd.DataFrame({
        'Track Name': [input_song_name],
        'Artists': [music_df.loc[music_df['Track Name'] == input_song_name, 'Artists'].values[0]],
        'Album Name': [music_df.loc[music_df['Track Name'] == input_song_name, 'Album Name'].values[0]],
        'Release Date': [music_df.loc[music_df['Track Name'] == input_song_name, 'Release Date'].values[0]],
        'Popularity': [weighted_popularity_score]
    })

    hybrid_recommendations = pd.concat([content_based_rec, input_song_df], ignore_index=True)
    

    # Sort the hybrid recommendations based on weighted popularity score
    hybrid_recommendations = hybrid_recommendations.sort_values(by='Popularity', ascending=False)

    # Remove the input song from the recommendations
    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['Track Name'] != input_song_name]


    return hybrid_recommendations

In [65]:
#Getting Reccommendation for Fortnight (feat. Post Malone) by Taylor Swift
input_song_name = "Fortnight (feat. Post Malone)"
recommendations = hybrid_recommendations(input_song_name, num_recommendations=5)
print(f"Hybrid recommended songs for '{input_song_name}':")
recommendations.head()

Hybrid recommended songs for 'Fortnight (feat. Post Malone)':


,Track Name,Artists,Album Name,Release Date,Popularity
0,we can't be friends (wait for your love),Ariana Grande,eternal sunshine,2024-03-08,95.0
3,Saturn,SZA,Saturn,2024-02-22,93.0
2,Madonna,"Natanael Cano, Oscar Maydon",Madonna,2024-03-08,92.0
4,Beautiful Things,Benson Boone,Fireworks & Rollerblades,2024-04-05,89.0
1,Down Bad,Taylor Swift,THE TORTURED POETS DEPARTMENT,2024-04-18,88.0
